In [1]:
!pip install pandas 
!pip install sklearn

In [2]:
import numpy as np
import pandas as pd
import os,datetime

import tensorflow as tf
from tensorboard import notebook



In [4]:
###---< Enable Tensorboard in port 6666 >---###
%load_ext tensorboard
log_dir="/tf/logs"
%tensorboard --logdir {log_dir} --bind_all   #always --bind_all
callback=tf.keras.callbacks.TensorBoard(log_dir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 39), started 1:00:15 ago. (Use '!kill 39' to kill it.)

In [97]:
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir /tf/logs (started 0:34:48 ago; pid 39)


In [76]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))


8192/2194 [================================================================================================================] - 0s 0us/step
Local copy of the dataset file: /root/.keras/datasets/iris_training.csv


In [79]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']


Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


In [80]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)


In [81]:
features, labels = next(iter(train_dataset))

print(features)


OrderedDict([('sepal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([6.6, 6.3, 7. , 6.7, 7.7, 6.1, 6.7, 6.1, 7.3, 6.7, 4.4, 5.4, 5.1,
       5.6, 5.8, 5.4, 5.4, 6.4, 7.7, 7.7, 5.9, 6.2, 5. , 6.7, 6.5, 5.9,
       6.3, 4.6, 5.8, 5.8, 7.7, 6.5], dtype=float32)>), ('sepal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([2.9, 2.5, 3.2, 3.3, 2.8, 2.8, 3. , 3. , 2.9, 3.1, 3.2, 3.9, 3.5,
       2.7, 2.7, 3.7, 3.9, 2.7, 2.6, 3.8, 3.2, 2.2, 2. , 3.1, 3. , 3. ,
       3.3, 3.4, 2.6, 2.7, 3. , 3. ], dtype=float32)>), ('petal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([4.6, 5. , 4.7, 5.7, 6.7, 4.7, 5. , 4.9, 6.3, 5.6, 1.3, 1.3, 1.4,
       4.2, 4.1, 1.5, 1.7, 5.3, 6.9, 6.7, 4.8, 4.5, 3.5, 4.4, 5.2, 5.1,
       4.7, 1.4, 4. , 5.1, 6.1, 5.8], dtype=float32)>), ('petal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([1.3, 1.9, 1.4, 2.1, 2. , 1.2, 1.7, 1.8, 1.8, 2.4, 0.2, 0.4, 0.3,
       1.3, 1. , 0.2, 0.4, 1.9, 2.3, 2.2, 1.8, 1.5, 1. , 1.4,

In [82]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

In [83]:
train_dataset = train_dataset.map(pack_features_vector)


In [84]:
features, labels = next(iter(train_dataset))

print(features[:5])


tf.Tensor(
[[5.  3.5 1.6 0.6]
 [6.9 3.2 5.7 2.3]
 [4.8 3.  1.4 0.1]
 [5.8 2.7 5.1 1.9]
 [7.2 3.2 6.  1.8]], shape=(5, 4), dtype=float32)


In [85]:
def create_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
      tf.keras.layers.Dense(10, activation=tf.nn.relu),
      tf.keras.layers.Dense(3)
    ])
    return model
model = create_model()

In [86]:
predictions = model(features)
predictions[:5]


<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-0.6698812 ,  3.0292413 ,  1.4628589 ],
       [-0.29717648,  5.807516  ,  3.1580958 ],
       [-0.5405931 ,  2.6424017 ,  1.2552483 ],
       [-0.15525007,  4.935645  ,  2.7362504 ],
       [-0.13413274,  5.8003983 ,  3.1542149 ]], dtype=float32)>

In [87]:
tf.nn.softmax(predictions[:5])


<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.02006027, 0.8106721 , 0.16926765],
       [0.00208064, 0.93203205, 0.06588738],
       [0.03210951, 0.77444535, 0.19344509],
       [0.00550795, 0.8952369 , 0.09925514],
       [0.00246511, 0.9314735 , 0.06606138]], dtype=float32)>

In [88]:
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))
print("    Labels: {}".format(labels))


Prediction: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
    Labels: [0 2 0 2 2 0 0 0 1 0 2 1 1 0 2 2 0 2 0 0 1 0 2 2 0 0 1 0 1 2 1 0]


In [89]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [91]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)

l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))


Loss test: 2.5883514881134033


In [92]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)


In [94]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))


Step: 0, Initial Loss: 2.5883514881134033
Step: 1,         Loss: 2.0027570724487305


In [95]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))


Epoch 000: Loss: 1.448, Accuracy: 37.500%
Epoch 050: Loss: 0.749, Accuracy: 74.167%
Epoch 100: Loss: 0.487, Accuracy: 86.667%
Epoch 150: Loss: 0.298, Accuracy: 95.833%
Epoch 200: Loss: 0.215, Accuracy: 96.667%
